# Overview of Cryptographic Primitives

In this section we cover the basics of the cryptographic primitives that can be used for cryptocurrencies. We specially cover hashing algorithms and the asymmetric cryptography that is used for signatures.

## Hashing Algorithms

The goal is to have a long message as input and produce an output which is much shorter called the hash or message digest. Furthermore, we want it to have certain properties.

- **Input**: long message
- **Output**: short fixed size block (called hash or message digest)
 
*We want the same input always produces the same output (deterministic)*

## *<font color=" #6495ED">Exercise</font>*

 - A *non* security related example of hash functions?
 - A security related example of hash functions?

*** There is a difference between hash function and cryptographic hash functions ***

### Desired properties
  - Pre-image: Given a hash *h* it is computationally infeasible to find a message *m* that produces *h*
  - Second preimage: Given message m, it is computationally infeasible to find a message m’, (m ≠ m’) such that, h(m) = h(m’)
  - Collisions: It is computationally difficult to find any two messages m, m’ (m ≠ m’) such that, h(m) = h(m’)
  

**Examples**:
 - Recommended Hash Algorithm (SHA-2, SHA-3) by NIST
 - RIPEMD
 - <strike>SHA-1</strike>: output 160 bits being phased out (shattered.io)
 - <strike>MD2, MD4, and MD5</strike> by Ron Rivest [RFC1319, 1320, 1321]
 

### SHA Family

Secure Hash Algorithm (SHA) family, is a series of hashing algorithms. Ranging from SHA-0 to SHA-3. SHA-0 should never be used, it's advised to move from SHA-1 to SHA-2. SHA-3 is the most recent version, published in 2015.

 * SHA-1: Digest size (160), Block size (512)
 * SHA-2: Digest size (224, 256, 384, or 512), Block size (512, 1024)
 * SHA-3: Digest size (224, 256, 384, 512), Block size (1600)

### RIPEMD (RACE Integrity Primitives Evaluation Message Digest)

Another family of hashing functions. Based on the design principles of MD4. Bitcoin used the RIPEMD-160, the 160 bit version of the hashing family.

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64 # to produce human readable encoding of the bytes

digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"PyCon")
digest.update(b"2018")
msg_digest = digest.finalize()
# Notice the output size of the digest
print ("msg_digest:", len(msg_digest), len(msg_digest) * 8)
print ("base64 encoding:", base64.b64encode(msg_digest))

print()

digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"PyCon2018")
msg_digest = digest.finalize()
# Notice the output size of the digest
print ("msg_digest:", len(msg_digest), len(msg_digest) * 8)
print ("base64 encoding:", base64.b64encode(msg_digest))

In [ ]:
import hashlib

print(hashlib.algorithms_available)

In [ ]:
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(b"PyCon 2018")
msg_digest = ripemd160.digest()
print ("msg_digest:", len(msg_digest), len(msg_digest) * 8)
print ("base64 encoding:", base64.b64encode(msg_digest))

## *<font color=" #6495ED">Exercise</font> *
 - Change "PyCon2018", to "PyCon 2018" and see check the result

In [ ]:
## SOLUTION



Since the hashes are random, we can predict another hash of a value based on previous outputs. Therefore, to find a specific hash we need to bruteforce all possible input values, to find our desired output. In other words, the previous work that we have done, won't help us in the future problems.

## *<font color=" #6495ED">Exercise</font> *
 - Find a has value that starts with two consecutive zeros ('00')
 - Plot the number of tries it takes for a hash with n zeros (n = {1,2,3,4,5}), based on the average of 5 runs.

In [ ]:
## SOLUTION



In [ ]:
## SOLUTION



In [ ]:
## SOLUTION



## Different Hashing Algorithms

Bitcoin uses SHA-256 and RIPEMD-160. However, there many are other hashing families, some of which are used by other cryptocurrencies. At the core all cryptographically secure hashing functions have the same three features that we mentioned before, some variant have extra features. For example, there are variant that are slower/faster, use more memory.

 - scrypt (Litecoin, Dogecoin)
 - CryptoNight (Monero)
 - X11 (DASH)

## Asymmetric Encryption Simplified

Reminder: In the asymmetric encryption schemes the parties use ***different*** keys, that are mathematically ***related*** to each other.

## *<font color=" #6495ED">Exercise</font>*

 - Why asymmetric encryption is useful?
 - Give a few examples where it can be used?

## Elliptic-curve cryptography (ECC) 

An approach to public key cryptography using elliptical curves over finite fields. Believed to provide the same level of security by having smaller key size.

Points on an elliptical curve over finite field that satisfy the following equation.
 
$y^2 = x^3+a.x+b$

### SECCP256k1

Standardized by Standards for Efficient Cryptography (SEC). Became significantly popular after being used in Bitcoin. $y^2 = x^3 + 7$
 
<img src="include/Secp256k1.png" alt="Secp256k1" style="width: 450px;"/>

source: https://en.bitcoin.it/wiki/Secp256k1

### Digital Signature Algorithm (DSA)

A signature algorithm, based on ElGamal. Private key is a large random number $x$, and public key in $y = g^x$ mod $p$.

To sign a message $m$, with parameters (p, q, g):

 - Generate random value $k$, $1 < k < q$
 - r = $(g ^ k$ mod $p$) mod $q$
 - s = $ k ^ {-1} (H(m) + xr)$ mod $q$
 - Signature is the tuple $(r,s)$
 
To verify a signature:
   
   - $w = s ^ {−1}$ mod $q$
   - $u_1 = H (m) \cdot w$ mod $q$
   - $u_2 = r \cdot w$ mod $q$
   - $v = ( g ^ {u_1} y^{u_2}$ mod $p$ ) mod $q$
   - $v = r$, $iff$ siganture is valid

### Elliptic Curve Digital Signature Algorithm (ECDSA)

DSA just with the EC. Replace exponentiation with scalar point multiplication (over simplification)

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
private_key = ec.generate_private_key(ec.SECP256K1(), default_backend())
data = b"data to be signed"
signature = private_key.sign(data, ec.ECDSA(hashes.SHA256()))

In [ ]:
public_key = private_key.public_key()
public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))

## *<font color=" #6495ED">Exercise</font>*
 - Change the signature to some random value
 - Change data to some random value
 - Change the hashing value

In [ ]:
# SOLUTION


In [ ]:
# SOLUTION


In [ ]:
# SOLUTION


In [ ]:
# SOLUTION
